In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)
KTF.set_session(session)
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, Flatten, Input, concatenate
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Nadam
import keras.backend as K
import numpy as np
import pandas as pd
import pickle




/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def zuhe(line):
    line['hour'] = line['hour'].astype(str)
    line['weekday'] = line['weekday'].astype(str)
    line['is_peek'] = line['is_peek'].astype(str)
    line['is_workday'] = line['is_workday'].astype(str)
                                   
    line['hour_weekday'] = line['hour'] + line['weekday']
    line['peak_workday'] = line['is_peek'] + line['is_workday']
    
    line['hour'] = line['hour'].astype(int)
    line['is_peek'] = line['is_peek'].astype(int)
    line['is_workday'] = line['is_workday'].astype(int)
    line['weekday'] = line['weekday'].astype(int)
    
    line['hour_weekday'] = line['hour_weekday'].astype(int)
    line['peak_workday'] = line['peak_workday'].astype(int)
    return line

In [11]:
with open('./data/train-id4-crowd-grid4.txt', 'rb') as data_file:
    train = pickle.load(data_file)
    #训练验证都划分
    #900 29 26 #800 27.9 25.9 #700 27.5 25.3 #27.4 24.8 #26.4 24.08 #23 21 #(90)250 21 19.5 #(80)17. 16.5
    #训练划分 验证不划分
    #250
train = train[(train['Diff_Time']<600)]
train = zuhe(train)
with open('./data/test-id4-crowd-grid4.txt', 'rb') as data_file:
    test = pickle.load(data_file)
test = zuhe(test)
from sklearn import preprocessing
x1 = train['ID'].as_matrix().reshape([-1,1])
N1 = x1.shape[0]

x2 = test['ID'].as_matrix().reshape([-1,1])
N2 = x2.shape[0]
x = np.concatenate((x1,x2))
x = preprocessing.LabelEncoder().fit_transform(x) #13963746x21567
print(x.shape)#(14454126, 21567)
train['new_ID'] = x[:N1]
test['new_ID'] = x[N1:]

/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/kawayi-4/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(14223933,)


In [12]:
col1 = ['new_ID']
col2 = [c for c in train if
       c not in ['Unnamed: 0','ID2', 's_ij', 'e_ij', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']

X_train1 = train[col1].values
X_train2 = train[col2].values
X_train3 = train[col3].values
X_train4 = train[col4].values
y_train = train['Diff_Time'].values

In [13]:
col1 = ['new_ID']
col2 = [c for c in test if
       c not in ['Unnamed: 0','ID2', 's_ij', 'e_ij', 's_x', 's_y', 'e_x', 'e_y', 'new_ID','ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'O_TIME', 'aver_v', 'max_v',
                 'Diff_Time','Distance1', 'distance2','TERMINALNO', 'new_dist']]
col3 = ['Source_Station_encode']
col4 = ['Target_Station_encode']

print(col1,col2,col3,col4)
X_test1 = test[col1].values
X_test2 = test[col2].values
X_test3 = test[col3].values
X_test4 = test[col4].values
print('test',X_test1.shape,X_test2.shape,X_test3.shape,X_test4.shape)

['new_ID'] ['Distance', 'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'h_aver_diff', 'h_aver_d', 'h_aver_v', 'is_crowd', 'grid_aver_diff', 'grid_aver_d', 'Source_Station_encode', 'Target_Station_encode', 'hour_weekday', 'peak_workday'] ['Source_Station_encode'] ['Target_Station_encode']
test (490380, 1) (490380, 20) (490380, 1) (490380, 1)


In [47]:
from keras.layers import *
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean((y_pred - y_true)**2))
    
def mse_loss(y_true,y_pred):
    return K.sqrt(K.mean((y_pred - y_true)**2))

def get_model():
    input1 = Input(shape=(1,))
    input2 = Input(shape=(20,))
    input3 = Input(shape=(1,))
    input4 = Input(shape=(1,))

    x1 = Embedding(21064, 128, input_length=1)(input1)
    x1 = Flatten()(x1)
    x1 = BatchNormalization()(x1)
    
    x2 = BatchNormalization()(input2)
    x2 = Dense(units=128,activation='relu')(x2)
    
    x3 = Embedding(4609, 48, input_length=1)(input3)
    x3 = BatchNormalization()(x3)
    x3 = Flatten()(x3)
    
    x4 = Embedding(4609, 48, input_length=1)(input4)
    x4 = BatchNormalization()(x4)
    x4 = Flatten()(x4)

    def dist(x):
        return x[0]*x[1]/(K.sum(x[0]**2,axis=1,keepdims=True)+K.sum(x[1]**2,axis=1,keepdims=True))    
    #     Aggregate
    x3_x4 = Lambda(dist)([x3,x4])

    x = concatenate([x1, x2, x3, x4, x3_x4])
#     x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation='relu')(x)
    
    model = Model(inputs=[input1, input2, input3, input4], outputs=out)
    print(model.summary())
    model.compile(loss=mse_loss, optimizer=Nadam(), metrics=[root_mean_squared_error])
    return model


In [48]:
from sklearn.model_selection import StratifiedKFold, KFold

def get_stacking(X_train1, X_train2, X_train3, X_train4, labels, n_folds=10):
    train_num, test_num = X_train1.shape[0], X_test1.shape[0]
    second_level_train_set = np.zeros((train_num,))
    second_level_test_set = np.zeros((test_num,))
    test_nfolds_sets = np.zeros((test_num, n_folds))
    
    folds = KFold(n_splits=n_folds, shuffle=True, random_state=1001001)
        
    for n_fold,(train_idx, val_idx) in enumerate(folds.split(labels)):
        train_1 = X_train1[train_idx]
        train_2 = X_train2[train_idx]
        train_3 = X_train3[train_idx]
        train_4 = X_train4[train_idx]
        train_y = labels[train_idx]

        val_1 = X_train1[val_idx]
        val_2 = X_train2[val_idx]
        val_3 = X_train3[val_idx]
        val_4 = X_train4[val_idx]
        val_y = labels[val_idx]
#         ckpt_path = './logtianchi/'+ model_list[n_fold]
#         print(ckpt_path)
        model = get_model()
        if n_fold == 0:
            print(model.summary())
        print(n_fold)
        


        ckpt_path = './log/qrfcv_'+str(n_fold)+'_weights-{val_loss:.4f}.hdf5'
        
        for i in range(10):
            model.fit([train_1,train_2,train_3,train_4],train_y+np.random.uniform(-1,1,(len(train_y,))), 
                      validation_data=([val_1,val_2,val_3,val_4],val_y), 
                      epochs=1, 
                      batch_size=1024,
    #                   callbacks=[
    #                       ModelCheckpoint(ckpt_path,monitor='val_loss',verbose=1,save_best_only=True,mode='min'),
    #                       EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='auto')]
                     )
        
        second_level_train_set[val_idx] = model.predict([val_1, val_2,val_3,val_4], batch_size=1024)[:, 0]
        test_nfolds_sets[:,n_fold] = model.predict([X_test1, X_test2, X_test3, X_test4], batch_size=1024)[:, 0]
    
    "---save---"
    with open('./data/qrf_test_nfolds_sets3.txt', 'wb') as data_file:
        pickle.dump(test_nfolds_sets, data_file)

    second_level_test_set[:] = test_nfolds_sets.mean(axis=1)

    result = second_level_test_set
    
    return second_level_train_set, second_level_test_set, result

In [ ]:
train_sets = []
test_sets = []
train_set, test_set, result = get_stacking(X_train1, X_train2, X_train3, X_train4, y_train, n_folds=10)
train_sets.append(train_set)
test_sets.append(test_set)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_55 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_56 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_53 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_41 (Embedding)        (None, 1, 48)        221232      input_55[0][0]                   
__________________________________________________________________________________________________
embedding_

Train on 12360197 samples, validate on 1373356 samples
Epoch 1/10
12360197/12360197 [==============================] - 471s 38us/step - loss: 39.7314 - root_mean_squared_error: 39.7314 - val_loss: 37.4213 - val_root_mean_squared_error: 37.4213
Epoch 2/10
12360197/12360197 [==============================] - 464s 38us/step - loss: 37.9178 - root_mean_squared_error: 37.9178 - val_loss: 36.7210 - val_root_mean_squared_error: 36.7210
Epoch 3/10
12360197/12360197 [==============================] - 464s 38us/step - loss: 37.2656 - root_mean_squared_error: 37.2656 - val_loss: 36.7574 - val_root_mean_squared_error: 36.7574
Epoch 4/10
12360197/12360197 [==============================] - 462s 37us/step - loss: 36.8045 - root_mean_squared_error: 36.8045 - val_loss: 36.1791 - val_root_mean_squared_error: 36.1791
Epoch 5/10
12360197/12360197 [==============================] - 460s 37us/step - loss: 36.4402 - root_mean_squared_error: 36.4402 - val_loss: 36.7348 - val_root_mean_squared_error: 36.7348


12360197/12360197 [==============================] - 463s 37us/step - loss: 35.7968 - root_mean_squared_error: 35.7968 - val_loss: 37.1291 - val_root_mean_squared_error: 37.1291
Epoch 9/10
12360197/12360197 [==============================] - 259s 21us/step - loss: 35.5974 - root_mean_squared_error: 35.5974 - val_loss: 38.2429 - val_root_mean_squared_error: 38.2429
Epoch 10/10
11553792/12360197 [===========================>..] - ETA: 14s - loss: 35.3909 - root_mean_squared_error: 35.3909